<a href="https://colab.research.google.com/github/shinhs0920/19-lab/blob/master/keras__mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fashion_mnist : MLP

In [0]:
from __future__ import print_function 
 
 
# Keras 실행하기
import keras  
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.datasets import fashion_mnist 
 
 
# 데이터 불러오기
import numpy as np 
import os 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
 
 
# 데이터를 훈련,평가 데이터로 나누기 / * 전처리를 데이터 불러오면서 동시에 할 수 없음
x_train, y_train, x_valid, y_valid, x_test ,y_test = [], [], [], [], [], [] 
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data() 
x_train = x_train.reshape(60000,28,28) 
x_test = x_test.reshape(10000,28,28) 

 
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2) 
 
# 데이터 정규화 / * 정규화 방법 차이 설명 
x_train = x_train.astype('float32') 
x_valid = x_valid.astype('float32') 
x_test = x_test.astype('float32') 
x_train /= 255 
x_valid /= 255 
x_test /= 255 
 
 
x_train = x_train.reshape(len(x_train),28,28) 
x_test = x_test.reshape(10000,28,28) 
x_valid = x_valid.reshape(len(x_valid),28,28) 
 
# 원-핫 벡터로 변환 / * 케라스는 0과1 , 파이토치는 특정 값
y_train = keras.utils.to_categorical(y_train, 10) 
y_valid = keras.utils.to_categorical(y_valid, 10) 
y_test = keras.utils.to_categorical(y_test, 10) 

 
# 모델 구축 : Flatten - Dense - Dropout - Softmax - SGD 최적화 / * 케라스 : 층을 각각 코드로 추가하여 모델을 구성해야함
model = Sequential() 
model.add(Flatten(input_shape=(28,28))) 
model.add(Dense(256, activation='relu', kernel_initializer = 'he_normal' ,input_shape=(28*28,))) 
model.add(Dropout(0.2)) 
model.add(Dense(128, activation='relu',kernel_initializer = 'he_normal')) 
model.add(Dropout(0.2)) 
model.add(Dense(10, activation='softmax',kernel_initializer = 'he_normal')) 
optim = keras.optimizers.SGD(lr=0.01) 
 
# 최적화 방식 설명 
model.compile(loss='categorical_crossentropy', 
			  optimizer=optim, 
 			  metrics=['accuracy']) 
 
# 모델 훈련, 검증 
history = model.fit(x_train, y_train,
 					 batch_size=64, 
 					 epochs=50, 
 					 verbose=2, 
 					 validation_data=(x_valid, y_valid)) 
 
 
print(model.summary()) 

# 모델 평가  	 
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test loss:', score[0]) 
print('Test top 1 accuracy:', score[1]) 



Train on 48000 samples, validate on 12000 samples
Epoch 1/50
 - 3s - loss: 1.0052 - acc: 0.6594 - val_loss: 0.6206 - val_acc: 0.7989
Epoch 2/50
 - 3s - loss: 0.6670 - acc: 0.7718 - val_loss: 0.5301 - val_acc: 0.8192
Epoch 3/50
 - 3s - loss: 0.5874 - acc: 0.7988 - val_loss: 0.4846 - val_acc: 0.8342
Epoch 4/50
 - 3s - loss: 0.5415 - acc: 0.8142 - val_loss: 0.4579 - val_acc: 0.8421
Epoch 5/50
 - 3s - loss: 0.5116 - acc: 0.8210 - val_loss: 0.4407 - val_acc: 0.8446
Epoch 6/50
 - 3s - loss: 0.4883 - acc: 0.8287 - val_loss: 0.4268 - val_acc: 0.8466
Epoch 7/50
 - 3s - loss: 0.4731 - acc: 0.8341 - val_loss: 0.4150 - val_acc: 0.8522
Epoch 8/50
 - 3s - loss: 0.4562 - acc: 0.8414 - val_loss: 0.4047 - val_acc: 0.8543
Epoch 9/50
 - 3s - loss: 0.4446 - acc: 0.8447 - val_loss: 0.3950 - val_acc: 0.8572
Epoch 10/50
 - 3s - loss: 0.4340 - acc: 0.8456 - val_loss: 0.3882 - val_acc: 0.8602
Epoch 11/50
 - 3s - loss: 0.4231 - acc: 0.8505 - val_loss: 0.3806 - val_acc: 0.8627
Epoch 12/50
 - 3s - loss: 0.4151 - 

# Fashion_Mnist : CNN

- CNN 모델<br>
  MLP 모델과는 달리 MNIST 이미지에 대해 입력 벡터 배신 입력 텐서를 갖는다. 따라서 훈련/데이터 이미지 크기를 재조정해야한다.
![대체 텍스트](https://github.com/syeong1218/keras-fig/blob/master/1.4.1.PNG?raw=true)


### 1. Conv2D 계층을 사용한다


- 커널<br> 전체 이미지를 움직이는 직사각형 패치
![대체 텍스트](https://github.com/syeong1218/keras-fig/blob/master/1.4.2.PNG?raw=true)
- 합성곱<br>  입력 이미지를 커널을 사용해 다른 행렬의 형태로 변형시키는 것. 
<br>이렇게 변형된 이미지를 특징 맵이라고 한다.
![대체 텍스트](https://github.com/syeong1218/keras-fig/blob/master/1.4.3.PNG?raw=true)


### 2. 폴링 연산

MaxPooling2D 계층은 각 특징 맵을 압축한다. 
![대체 텍스트](https://github.com/syeong1218/keras-fig/blob/master/1.4.4.PNG?raw=true)


### 3. Flatten

Conv2D 계층과 풀링 계층에서 나온 출력 값들을 텐서 형태에서 다시 행렬 형태로 바꾸어 준다.

In [9]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

# fashoin-mnist 데이터 불러오기
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# 학습 셋 크기(shape) - 이미지 크기가 28x28 인 60,000 개의 학습 이미지 데이터, 60,000 개의 레이블
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# 이미지 값 정규화
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# 학습 데이터 셋을 학습 / 평가 셋으로 나눈다. (# 학습 셋: 55,000, 검증 셋: 5000)
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# 입력 이미지의 크기를 (28, 28) 에서 (28, 28, 1) 로 배열 차원을 변경(reshape)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

# 레이블에 원-핫 인코딩 적용 
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 모델 구현 / *간단 설명  - input_shape 형태 (텐서), 활성화함수 컨볼루션 매서드에서 직접 지정 
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=10, ernel_size=5, padding='valid', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=5, padding='valid', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))


model.summary()

# 원-핫 벡터를 위한 손실 함수 / SGD 최적화 사용
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# 훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장
# 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행.
# Callbacks:  체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=40,
         validation_data=(x_valid, y_valid),
         callbacks=[checkpointer])

# 가장 높은 검증 정확도의 가중치 불러오기
model.load_weights('model.weights.best.hdf5')

# 테스트 셋을 통해 모델 평가
score = model.evaluate(x_test, y_test, verbose=0)

# 테스트 정확도
print('\n', 'Test accuracy:', score[1])


x_train shape: (60000, 28, 28) y_train shape: (60000,)
x_train shape: (60000, 28, 28) y_train shape: (60000,)
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 24, 24, 10)        260       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 10)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 20)          5020      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 4, 20)          0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 4, 4, 20)          0         
__________

# Fashion_Mnist : ResNet(VER.1)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# 훈련 매개변수
batch_size = 32
epochs = 200
data_augmentation = True
num_classes = 10

subtract_pixel_mean = True

# 모델 파라미터 - 모델 선정 방식
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# 모델 버전
# 최초 논문: version = 1 (ResNet v1), 
version = 1

# 제공된 모델 매개변수 n으로부터 계산된 네트워크 깊이
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

model_type = 'ResNet%dv%d' % (depth, version)

# CIFAR10 데이터 불러오기
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# input image dimensions.
input_shape = x_train.shape[1:]

# 정규화
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """ 학습률은 lr_schedule()을 통해 80, 120, 160, 180 epochs 에서 시작하여 점차 감소시킨다.
    기본값은 1e-3 이다. lr_schedule()함수는 모델 훈련동안 한 epoch 당 callbacks 변수로 호출된다.

    # Arguments
        에폭의 수 :-> epoch (int)
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    Arguments:
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    Returns:
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal', # 역전파 시 수렴될 수 있게 해주는 것
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')
    
    # 모델 정의 시작
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    
    # 잔차 유닛 인스턴스화
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
           
            if stack > 0 and res_block == 0:  
                strides = 2  
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            
            if stack > 0 and res_block == 0:
               
               # 변경된 차원을 맞추기 위해 잔차 숏컷 연결을 선형으로 사영 
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # 분류기 추가
    # v1은 마지막 숏컷 연결-ReLU 후에는 BN을 사용하지 않는다.
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # 모델 인스턴스화
    model = Model(inputs=inputs, outputs=outputs)
    return model

#ResNet v2 구현

def resnet_v2(input_shape, depth, num_classes=10):
   
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 110 in [b])')
   
   # 모델 정의
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
   # v2에서는 2 경로로 나뉘기 전에 입력에 BN-ReLU와 함께 Conv2D를 수행 
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                # first layer and first stage
                if res_block == 0:  
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
               
                if res_block == 0:
                   
                    strides = 2 

           
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
              
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])

        num_filters_in = num_filters_out

    # add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
plot_model(model, to_file="%s.png" % model_type, show_shapes=True)
print(model_type)

# prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # this will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, 
                        steps_per_epoch=len(x_train)//batch_size,
                        callbacks=callbacks)

# score trained model
scores = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

170500096/170498071 [==============================] - 11s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
______________________